# All pairs shortest path algorithm

Shortest paths between every pair of vertices -algorithm implemented by QUBO in quantum annealer. Classical version of this is Floyd-Warshall algorithm: https://en.wikipedia.org/wiki/Floyd%E2%80%93Warshall_algorithm

Here directed graph $G=(V,E)$ has sets of vertices $V$ and edges $E \subseteq \{(x,y)|(x,y) \in V^2 and x \not = y\}$. For every edge there is weight $w_{xy}$. Task is to find path with minimum sum of weights for every combination of $(s,t) \in E^2$.

In this solution QUBO has seperate elements for starting vertices $s \in V$ and traget vertices $t \in V$. In addtion it has one element for each edge $e \in E$. So alltogother QUBO is matrix of $(|V|+|V|+|E|) \times (|V|+|V|+|E|)$. First block is |V| elements which indicate what is the starting vertice $s$. Second block is |V| elements which indicate what is the terminating vertice $t$. Last block of $|E|$ indicates which edges form the shortest path between $s$ and $t$.

Penalty $p=\sum w_{xy}$.

Following constraints are build to QUBO: 
1. Exactly one s vertice (if more than one: $2p$)
2. Exactly one t vertice (if more than one: $2p$)
3. Vertices s and t are different (if not: $p$)
4. There should be one vertice starting from $s$ ($-p$ for each), no vertice should end to $s$ ($p$ for each)
5. There should be one vertice ending to $t$ ($-p$ for each), no vertice should start from $t$ ($p$ for each)
6. Two edges should not start/end to the same vertice, for example $s$ or $t$ (if so $2p$) 
7. Two edges should form a chain (ok chain gives $p$, otherwise more than that)
8. Path having lower weights should be prioritised

Proper path with correct $s$ and $t$ would give energy level of $w-p$ (constraint 7 brings minimum $p$ and constraint 4 and 5 both minimum $-p$). Because $p=\sum w_{xy}$ all samples with energy level below zero are correct paths. From this set of samples we choose lowest energy level sample for each $(s,t)$. 

In [1]:
import numpy as np
import time
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
from dwave.samplers import SimulatedAnnealingSampler
import dwave.inspector
import networkx as nx
from networkx.classes.function import path_weight
import random

## Function to create QUBO

E is array of tuples (1st vertice, 2nd vertice, weight), veritices is number of vertices, p is penalty.

In [2]:
def create_qubo(E,vertices,p):
    edges = len(E)
    Q = np.zeros((2*vertices + edges, 2*vertices + edges))

    t1 = time.time()

    # Constraints 1 and 2
    for i in range(vertices):
        for j in range(i+1,vertices):
            Q[i,j]=2*p
            Q[vertices+i,j+vertices] = 2*p
        
    # Constraint 3
    for i in range(vertices):
        Q[i,i+vertices] = p

    # Constraint 4
    for v in range(vertices):
        for i,e in enumerate(E):
            if e[0]==v:
                Q[v,vertices*2+i] = -p
            if e[1]==v:
                Q[v,vertices*2+i] = p

    # Constraint 5
    for v in range(vertices):
        for i,e in enumerate(E):
            if e[1]==v:
                Q[vertices+v,vertices*2+i] = -p
            if e[0]==v:
                Q[vertices+v,vertices*2+i] = p

    # Constraint 6
    for i in range(edges):
        for j in range(i+1,edges):
            if E[i][0]==E[j][0] or E[i][1]==E[j][1]:
                Q[vertices*2+i,vertices*2+j] = 2*p

    # Constraint 7
    for i in range(edges):
        Q[vertices*2+i,vertices*2+i] += p
        for j in range(i+1,edges):
            if E[i][1]==E[j][0] or E[i][0]==E[j][1]:
                Q[vertices*2+i,vertices*2+j] += -p

    # Constraint 8 
    for i in range(edges):
        Q[vertices*2+i,vertices*2+i] += E[i][2]
    
    return Q

## Some helper functions for getting results from sampleset

In [3]:
def path_from_sample(sample,E,vertices):
    s = 0
    t = 0
    w = 0
    for v in range(vertices):
        if sample['s'+str(v)]==1:
            s = v
        if sample['t'+str(v)]==1:
            t = v
    i = s
    path = [i]
    while i!=t:
        for e in E:
            if e[0]==i and sample[str(e[0]) + '-' + str(e[1])]==1:
                i = e[1]
                path.append(i)
                w += e[2]
    return (str(s)+'-'+str(t),path,w)
        

def result_info(sampleset, E, vertices):
    res = {}
    for s in sampleset.filter(lambda s: s.energy<0):
        st, path, w = path_from_sample(s,E,vertices)
        if st not in res:
            res[st]=(path,w)
    return res

def check_result(G,sampleset,E,vertices):
    ok = 0
    s = 0
    res = result_info(sampleset,E,vertices)
    for i in range(vertices):
        for j in range(vertices):
            if i!=j:
                if nx.has_path(G,i,j):
                    s += 1
                    p1 = [p for p in nx.all_shortest_paths(G,i,j,weight='weight')]
                    w = path_weight(G,p1[0],'weight')
                    if str(i)+'-'+str(j) in res.keys():
                        p2 = res[str(i)+'-'+str(j)]
                        if (not p2[0] in p1) and w!=p2[1]:
                            print('Path: '+str(p2[0])+' ('+str(p2[1])+'): correct: '+str(p1)+' ('+str(w)+')')
                        else:
                            ok += 1
                    else:
                        print('Path suggested: '+str(i)+'-'+str(j)+' missing: correct: '+str(p1)+' ('+str(w)+')')
    print('{:.0f}% ok'.format((100*ok/s)))

def make_G(E,vertices):
    G = nx.DiGraph()
    G.add_nodes_from([0, vertices-1])
    for e in E:
        G.add_edge(e[0], e[1], weight=e[2])
    return(G)

## Simple graph

### Define graph

Input graph is array of tuples (1st vertice, 2nd vertice, weight)

In [4]:
E1 = np.array([(0, 2, 1), (2, 1, 2), (1, 3, 3), (3, 2, 4), (0, 1, 5), (3, 4, 3), (2, 4, 8)])
vertices1 = 5
G1 = make_G(E1,vertices1)

Above graph visualised:

![](graph4.png)

### Max_count and labels

In [5]:
max_count1 = 0
edges = len(E1)
for e in E1:
    max_count1 += e[2]
print('Max count:',max_count1)

labels1 = {}
for i in range(vertices1):
    labels1[i]='s'+str(i)
    labels1[vertices1+i]='t'+str(i)   
for i,e in enumerate(E1):
    labels1[vertices1*2+i] = str(e[0]) + '-' + str(e[1])

Max count: 26


### Create QUBO and BQM

In [6]:
t1 = time.time()
Q1 = create_qubo(E1,vertices1,max_count1)
qubo_time = (time.time()-t1)*1000
print('Time used for construction Q (ms): {:.3f}\n'.format(qubo_time))

t1 = time.time()
bqm1 = dimod.BinaryQuadraticModel(Q1, 'BINARY')
bqm_time = (time.time()-t1)*1000
bqm1 = bqm1.relabel_variables(labels1, inplace=False)

Time used for construction Q (ms): 0.243



### Local deterministic solver

In [7]:
t1 = time.time()
sampleset = dimod.ExactSolver().sample(bqm1)
det_time = (time.time()-t1)*1000
print('Time used (ms): {:.3f}\n'.format(det_time))
print(sampleset.filter(lambda s: s.energy<0))

Time used (ms): 172.304

   0-1 0-2 1-3 2-1 2-4 3-2 3-4 s0 s1 s2 s3 s4 t0 t1 t2 t3 t4 energy num_oc.
0    0   1   0   0   0   0   0  1  0  0  0  0  0  0  1  0  0  -25.0       1
2    0   0   0   1   0   0   0  0  0  1  0  0  0  1  0  0  0  -24.0       1
1    0   1   0   1   0   0   0  1  0  0  0  0  0  1  0  0  0  -23.0       1
5    0   0   1   0   0   0   0  0  1  0  0  0  0  0  0  1  0  -23.0       1
16   0   0   0   0   0   0   1  0  0  0  1  0  0  0  0  0  1  -23.0       1
8    0   0   0   0   0   1   0  0  0  0  1  0  0  0  1  0  0  -22.0       1
3    0   0   1   1   0   0   0  0  0  1  0  0  0  0  0  1  0  -21.0       1
11   1   0   0   0   0   0   0  1  0  0  0  0  0  1  0  0  0  -21.0       1
4    0   1   1   1   0   0   0  1  0  0  0  0  0  0  0  1  0  -20.0       1
7    0   0   0   1   0   1   0  0  0  0  1  0  0  1  0  0  0  -20.0       1
13   0   0   1   0   0   0   1  0  1  0  0  0  0  0  0  0  1  -20.0       1
6    0   0   1   0   0   1   0  0  1  0  0  0  0  0  1  0  0  -

In [8]:
for k,v in result_info(sampleset, E1, vertices1).items():
    print('Route '+k+': '+str(v[0])+', weight '+str(v[1]))

Route 0-2: [0, 2], weight 1
Route 2-1: [2, 1], weight 2
Route 0-1: [0, 2, 1], weight 3
Route 1-3: [1, 3], weight 3
Route 3-4: [3, 4], weight 3
Route 3-2: [3, 2], weight 4
Route 2-3: [2, 1, 3], weight 5
Route 0-3: [0, 2, 1, 3], weight 6
Route 3-1: [3, 2, 1], weight 6
Route 1-4: [1, 3, 4], weight 6
Route 1-2: [1, 3, 2], weight 7
Route 2-4: [2, 1, 3, 4], weight 8
Route 0-4: [0, 2, 1, 3, 4], weight 9


In [9]:
check_result(G1,sampleset,E1,vertices1)

100% ok


### Local heuristic classical solver

In [10]:
num_reads = 5000
t1 = time.time()
sampleset2 = SimulatedAnnealingSampler().sample(bqm1, num_reads=num_reads).aggregate()
heur_time = (time.time()-t1)*1000
print('Time used (ms): {:.3f}\n'.format(heur_time))
print(sampleset2.filter(lambda s: s.energy<0))

Time used (ms): 1528.206

   0-1 0-2 1-3 2-1 2-4 3-2 3-4 s0 s1 s2 s3 s4 t0 t1 t2 t3 t4 energy num_oc.
14   0   1   0   0   0   0   0  1  0  0  0  0  0  0  1  0  0  -25.0     637
1    0   0   0   1   0   0   0  0  0  1  0  0  0  1  0  0  0  -24.0     546
2    0   0   1   0   0   0   0  0  1  0  0  0  0  0  0  1  0  -23.0     453
4    0   1   0   1   0   0   0  1  0  0  0  0  0  1  0  0  0  -23.0     409
5    0   0   0   0   0   0   1  0  0  0  1  0  0  0  0  0  1  -23.0     415
0    0   0   0   0   0   1   0  0  0  0  1  0  0  0  1  0  0  -22.0     347
3    0   0   1   1   0   0   0  0  0  1  0  0  0  0  0  1  0  -21.0     244
9    1   0   0   0   0   0   0  1  0  0  0  0  0  1  0  0  0  -21.0     261
6    0   0   0   1   0   1   0  0  0  0  1  0  0  1  0  0  0  -20.0     224
11   0   0   1   0   0   0   1  0  1  0  0  0  0  0  0  0  1  -20.0     264
16   0   1   1   1   0   0   0  1  0  0  0  0  0  0  0  1  0  -20.0     217
15   0   0   1   0   0   1   0  0  1  0  0  0  0  0  1  0  0  

In [11]:
for k,v in result_info(sampleset2,E1,vertices1).items():
    print('Route '+k+': '+str(v[0])+', weight '+str(v[1]))

Route 0-2: [0, 2], weight 1
Route 2-1: [2, 1], weight 2
Route 1-3: [1, 3], weight 3
Route 0-1: [0, 2, 1], weight 3
Route 3-4: [3, 4], weight 3
Route 3-2: [3, 2], weight 4
Route 2-3: [2, 1, 3], weight 5
Route 3-1: [3, 2, 1], weight 6
Route 1-4: [1, 3, 4], weight 6
Route 0-3: [0, 2, 1, 3], weight 6
Route 1-2: [1, 3, 2], weight 7
Route 2-4: [2, 1, 3, 4], weight 8
Route 0-4: [0, 2, 4], weight 9


In [12]:
check_result(G1,sampleset2,E1,vertices1)

100% ok


### Quantum solver

In [13]:
machine = DWaveSampler(solver={'chip_id': 'Advantage_system4.1'})
print('Chip:', machine.properties['chip_id'])
print('Qubits:', machine.properties['num_qubits'])

Chip: Advantage_system4.1
Qubits: 5760


In [14]:
num_reads = 500
sampleset3 = EmbeddingComposite(machine).sample(bqm1, num_reads=num_reads)

In [15]:
qpu_time = sampleset3.info['timing']['qpu_access_time'] / 1000
qubits = sum(len(x) for x in sampleset3.info['embedding_context']['embedding'].values())
print('QPU time used (ms): {:.1f}'.format(qpu_time))
print('Physical qubits used: {}\n'.format(qubits))
print(sampleset3.filter(lambda s: s.energy<0))

QPU time used (ms): 82.3
Physical qubits used: 33

   0-1 0-2 1-3 2-1 2-4 3-2 3-4 s0 s1 s2 s3 s4 t0 t1 t2 t3 t4 energy num_oc. ...
0    0   1   0   0   0   0   0  1  0  0  0  0  0  0  1  0  0  -25.0      18 ...
1    0   0   0   1   0   0   0  0  0  1  0  0  0  1  0  0  0  -24.0      18 ...
2    0   0   0   0   0   0   1  0  0  0  1  0  0  0  0  0  1  -23.0       4 ...
3    0   1   0   1   0   0   0  1  0  0  0  0  0  1  0  0  0  -23.0      17 ...
4    0   0   1   0   0   0   0  0  1  0  0  0  0  0  0  1  0  -23.0       6 ...
5    0   0   0   0   0   1   0  0  0  0  1  0  0  0  1  0  0  -22.0      13 ...
6    1   0   0   0   0   0   0  1  0  0  0  0  0  1  0  0  0  -21.0      12 ...
7    0   0   1   1   0   0   0  0  0  1  0  0  0  0  0  1  0  -21.0      15 ...
8    0   0   0   1   0   1   0  0  0  0  1  0  0  1  0  0  0  -20.0      18 ...
9    0   0   1   0   0   0   1  0  1  0  0  0  0  0  0  0  1  -20.0      11 ...
10   0   1   1   1   0   0   0  1  0  0  0  0  0  0  0  1  0  -20.0  

In [16]:
for k,v in result_info(sampleset3,E1,vertices1).items():
    print('Route '+k+': '+str(v[0])+', weight '+str(v[1]))

Route 0-2: [0, 2], weight 1
Route 2-1: [2, 1], weight 2
Route 3-4: [3, 4], weight 3
Route 0-1: [0, 2, 1], weight 3
Route 1-3: [1, 3], weight 3
Route 3-2: [3, 2], weight 4
Route 2-3: [2, 1, 3], weight 5
Route 1-4: [1, 3, 4], weight 6
Route 0-3: [0, 2, 1, 3], weight 6
Route 3-1: [3, 2, 1], weight 6
Route 1-2: [1, 3, 2], weight 7
Route 2-4: [2, 1, 3, 4], weight 8
Route 0-4: [0, 2, 4], weight 9


In [17]:
check_result(G1,sampleset3,E1,vertices1)

100% ok


### Timings

In [18]:
print('Construting QUBO: {:.3f}'.format(qubo_time))
print('Construting BQM: {:.3f}'.format(bqm_time))
print('\nLocal deterministic solver: {:.1f}'.format(det_time))
print('Local heuristic solver: {:.1f}'.format(heur_time))
print('Quantum solver: {:.1f}'.format(qpu_time))

Construting QUBO: 0.243
Construting BQM: 0.547

Local deterministic solver: 172.3
Local heuristic solver: 1528.2
Quantum solver: 82.3


## More complex graph

In [19]:
seed = 42
vertices2 = 10
random.seed(seed)
G2 = nx.gnp_random_graph(vertices2, 0.30, seed, directed=True)
nx.set_edge_attributes(G2, {e: {'weight': random.randint(1, 10)} for e in G2.edges})

In [30]:
E2 = [] 
for e in G2.edges(data=True):
    E2.append((e[0],e[1],e[2]['weight']))
print('Number of edges:',len(E2))
print('Number of vertices:',vertices2)

max_count2 = 0
for e in E2:
    max_count2 += e[2]
print('Max count:',max_count2)

labels2 = {}
for i in range(vertices2):
    labels2[i]='s'+str(i)
    labels2[vertices2+i]='t'+str(i)   
for i,e in enumerate(E2):
    labels2[vertices2*2+i] = str(e[0]) + '-' + str(e[1])

Number of edges: 34
Number of vertices: 10
Max count: 166


### QUBO and BQM

In [31]:
t1 = time.time()
p = max_count2
Q2 = create_qubo(E2,vertices2,p)
qubo_time = (time.time()-t1) * 1000
print('Time used for construction Q (ms): {:.3f}'.format(qubo_time))

t1 = time.time()
bqm2 = dimod.BinaryQuadraticModel(Q2, 'BINARY')
bqm_time = (time.time()-t1) * 1000
bqm2 = bqm2.relabel_variables(labels2, inplace=False)
print('Time used for construction BQM (ms): {:.3f}'.format(bqm_time))

Time used for construction Q (ms): 1.350
Time used for construction BQM (ms): 0.344


### Local heuristic solver

In [23]:
num_reads = 10000
t1 = time.time()
sampleset4 = SimulatedAnnealingSampler().sample(bqm2, num_reads=num_reads).aggregate()
heur_time = (time.time()-t1)*1000
print('Time used (ms): {:.3f}\n'.format(heur_time))

Time used (ms): 7515.466



In [24]:
check_result(G2,sampleset4,E2,vertices2)

Path: [6, 4, 7, 3] (16): correct: [[6, 2, 9, 0, 3]] (15)
99% ok


### Quantum solver

In [25]:
machine = DWaveSampler(solver={'chip_id': 'Advantage_system4.1'})
print('Chip:', machine.properties['chip_id'])
print('Qubits:', machine.properties['num_qubits'])

Chip: Advantage_system4.1
Qubits: 5760


In [26]:
num_reads = 1000
sampleset5 = EmbeddingComposite(machine).sample(bqm2, num_reads=num_reads)

In [27]:
qpu_time = sampleset5.info['timing']['qpu_access_time'] / 1000
qubits = sum(len(x) for x in sampleset5.info['embedding_context']['embedding'].values())
print('QPU time used (ms): {:.1f}'.format(qpu_time))
print('Physical qubits used: {}\n'.format(qubits))
#print(sampleset5)

QPU time used (ms): 153.2
Physical qubits used: 235



In [28]:
check_result(G2,sampleset5,E2,vertices2)

Path suggested: 0-1 missing: correct: [[0, 2, 1]] (12)
Path suggested: 0-2 missing: correct: [[0, 2]] (2)
Path suggested: 0-3 missing: correct: [[0, 3]] (1)
Path suggested: 0-4 missing: correct: [[0, 4]] (5)
Path suggested: 0-5 missing: correct: [[0, 2, 1, 5]] (21)
Path suggested: 0-6 missing: correct: [[0, 8, 6], [0, 4, 6]] (7)
Path suggested: 0-7 missing: correct: [[0, 2, 9, 7]] (6)
Path suggested: 0-8 missing: correct: [[0, 8]] (4)
Path suggested: 0-9 missing: correct: [[0, 2, 9]] (3)
Path suggested: 1-0 missing: correct: [[1, 0]] (4)
Path suggested: 1-2 missing: correct: [[1, 2]] (3)
Path suggested: 1-3 missing: correct: [[1, 0, 3]] (5)
Path suggested: 1-4 missing: correct: [[1, 4]] (2)
Path suggested: 1-5 missing: correct: [[1, 5]] (9)
Path suggested: 1-6 missing: correct: [[1, 4, 6]] (4)
Path suggested: 1-7 missing: correct: [[1, 4, 7]] (6)
Path suggested: 1-8 missing: correct: [[1, 8]] (2)
Path suggested: 1-9 missing: correct: [[1, 2, 9]] (4)
Path suggested: 2-0 missing: correct

### Timings

In [32]:
print('Construting QUBO (ms): {:.3f}'.format(qubo_time))
print('Construting BQM (ms): {:.3f}'.format(bqm_time))
print('\nLocal heuristic solver (ms): {:.1f}'.format(heur_time))
print('Quantum solver (ms): {:.1f}'.format(qpu_time))

Construting QUBO (ms): 1.350
Construting BQM (ms): 0.344

Local heuristic solver (ms): 7515.5
Quantum solver (ms): 153.2
